<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/tpu_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!gdown --id 1-8nsWLseynVj6Z9-E12w1ywnffnqJftm
!gdown --id 1UszbNYQnlNrAcPQkBwvb1wKX21oRPiqb

Downloading...
From: https://drive.google.com/uc?id=1-8nsWLseynVj6Z9-E12w1ywnffnqJftm
To: /content/Uembeds306Epochs.npy
2.59GB [00:21, 53.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UszbNYQnlNrAcPQkBwvb1wKX21oRPiqb
To: /content/Vembeds306Epochs.npy
2.59GB [00:43, 59.1MB/s]


In [61]:
%tensorflow_version 2.x
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import tensorflow as tf
from time import time
from tqdm import tqdm_notebook as tqdm
print('TensorFlow:', tf.__version__)

TensorFlow: 2.1.0-rc1


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  ['10.58.152.58:8470']
INFO:tensorflow:Initializing the TPU system: 10.58.152.58:8470


INFO:tensorflow:Initializing the TPU system: 10.58.152.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [3]:
workers = ['/job:worker/replica:0/task:0/device:TPU:'+str(i) for i in range(8)]
workers

['/job:worker/replica:0/task:0/device:TPU:0',
 '/job:worker/replica:0/task:0/device:TPU:1',
 '/job:worker/replica:0/task:0/device:TPU:2',
 '/job:worker/replica:0/task:0/device:TPU:3',
 '/job:worker/replica:0/task:0/device:TPU:4',
 '/job:worker/replica:0/task:0/device:TPU:5',
 '/job:worker/replica:0/task:0/device:TPU:6',
 '/job:worker/replica:0/task:0/device:TPU:7']

In [0]:
class Index:
    def __init__(self, u, v, worker):
        self.embeddings = tf.math.l2_normalize(u, axis=0) + tf.math.l2_normalize(v, axis=0)
        self.squared_norms_embeddings = tf.expand_dims(tf.square(tf.norm(self.embeddings, axis=1)), axis=0)
        self.worker = worker

    @tf.function
    def search(self, query_vector, top_k=None):
      with tf.device(worker):
        squared_norms_query_vector = tf.expand_dims(tf.square(tf.norm(query_vector, axis=1)), axis=0)
        dot_product = tf.reduce_sum(self.embeddings * query_vector, axis=1)
        distances = tf.maximum(self.squared_norms_embeddings + squared_norms_query_vector - 2 * dot_product, 0)
        sorted_indices =  tf.argsort(distances)
        if top_k:
            sorted_indices = sorted_indices[..., :top_k]
        nearest_distances = tf.reshape(tf.gather(distances[0], sorted_indices), shape=[-1, 1])
        return nearest_distances[..., 0], sorted_indices[0]

In [5]:
u_embeddings = np.load('Uembeds306Epochs.npy')
v_embeddings = np.load('Vembeds306Epochs.npy')
u_embeddings.shape, v_embeddings.shape

((1262996, 512), (1262996, 512))

In [0]:
# Discarding last 4 vectors to make number of vectors divisible by 8
u_embeddings = np.split(u_embeddings[:-4], 8, axis=0)
v_embeddings = np.split(v_embeddings[:-4], 8, axis=0)

In [122]:
## Place 1/8 of total embeddings on each TPU core
indices = []
for i, worker in enumerate(workers):
  with tf.device(worker):
    print('Building index on {}'.format(worker))
    indices.append(Index(u_embeddings[i], v_embeddings[i], worker))

Building index on /job:worker/replica:0/task:0/device:TPU:0
Building index on /job:worker/replica:0/task:0/device:TPU:1
Building index on /job:worker/replica:0/task:0/device:TPU:2
Building index on /job:worker/replica:0/task:0/device:TPU:3
Building index on /job:worker/replica:0/task:0/device:TPU:4
Building index on /job:worker/replica:0/task:0/device:TPU:5
Building index on /job:worker/replica:0/task:0/device:TPU:6
Building index on /job:worker/replica:0/task:0/device:TPU:7


In [0]:
def search(xq):
  D, I = [], []
  for i in range(8):
    print('Search running in index: {}'.format(indices[i].worker))
    d, idx = indices[i].search(xq, 1)
    D.append(d.numpy()[0])
    I.append(i*vecs_per_index + idx.numpy()[0])

  id_sorted = np.argsort(D)
  D = np.array(D)[id_sorted]
  I = np.array(I)[id_sorted]
  return D, I

In [128]:
vecs_per_index = 157874
n = 15500
split = 0 # [0, 7] . # Pick nth vector from given split
actual_n = vecs_per_index*split + n

xq = tf.nn.l2_normalize(u_embeddings[split][n]) + tf.nn.l2_normalize(v_embeddings[split][n])
xq = tf.reshape(xq, [1, -1])

s = time()
D, I = search(xq)
e = time()

print('Actual ID   :', actual_n)
print('Result ID   :', I[0], '')

print('Neighbors   :', I )
print('Distances   :', np.round(D, 4))
print('Time taken  :', np.round(e-s, 2), 'secs')

Search running in index: /job:worker/replica:0/task:0/device:TPU:0
Search running in index: /job:worker/replica:0/task:0/device:TPU:1
Search running in index: /job:worker/replica:0/task:0/device:TPU:2
Search running in index: /job:worker/replica:0/task:0/device:TPU:3
Search running in index: /job:worker/replica:0/task:0/device:TPU:4
Search running in index: /job:worker/replica:0/task:0/device:TPU:5
Search running in index: /job:worker/replica:0/task:0/device:TPU:6
Search running in index: /job:worker/replica:0/task:0/device:TPU:7
Actual ID   : 15500
Result ID   : 15500 
Neighbors   : [  15500  686470 1015284  257384  810155  630886 1123666  451210]
Distances   : [1.8499 1.8921 1.9137 1.924  1.9367 1.9453 1.9846 2.0654]
Time taken  : 0.24 secs


#### Checking accuracy

In [0]:
random_n = np.random.randint(0, vecs_per_index, 10000)
random_split = np.random.randint(0, 8, 10000)

In [134]:
y_true= []
y_pred = []
for n, split in tqdm(zip(random_n, random_split), total=10000):
  xq = tf.nn.l2_normalize(u_embeddings[split][n]) + tf.nn.l2_normalize(v_embeddings[split][n])
  xq = tf.reshape(xq, [1, -1])
  actual_n = vecs_per_index*split + n
  D, I = [], []
  for i in range(8):
    d, idx = indices[i].search(xq, 1)
    D.append(d.numpy()[0])
    I.append(i*vecs_per_index + idx.numpy()[0])
  id_sorted = np.argsort(D)
  y_pred.append(np.array(I)[id_sorted])
  y_true.append(actual_n)
y_true = np.array(y_true)
y_pred = np.array(y_pred)
print('Accuracy :', np.sum(y_true == y_pred[:, 0]) / 10000)

Accuracy : 0.6531
